# check oof df

In [1]:
import os
import gc
import yaml
import argparse

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append("/kaggle/src/dss_utils")
sys.path.append("/kaggle/src/exp")
sys.path.append("/kaggle/src/model")
sys.path.append("/kaggle/src/data")
from dss_metrics import score
from training_loop import get_valid_values_dict, concat_valid_input_info, get_oof_df, get_key_df
from dss_model import get_model
from dss_dataloader import get_loader


In [2]:
output_dir = "/kaggle/working"
exp_name = "exp022_dense_skffold_removesomeseries_epoch10"
folds = [0, 1, 2, 3, 4]
# folds = [0]
oof_df = pd.DataFrame()
for i in folds:
    print("fold", i)
    # df = pd.read_parquet(os.path.join(output_dir, "_oof", exp_name, f"raw_oof_df_fold{i}.parquet"))
    df = pd.read_parquet(os.path.join(output_dir, "_oof", exp_name, f"fold{i}_best_oof_df.parquet"))
    oof_df = pd.concat([oof_df, df], axis=0)


print(len(oof_df["series_id"].unique()))   


fold 0
fold 1
fold 2
fold 3
fold 4
269


In [3]:
oof_df = oof_df.sort_values(["series_id", "step"])
display(oof_df)


,series_id,step,anglez,enmo,second,minute,hour,date,anglez_mean_36,anglez_std_36,...,anglez_std_60,event,event_onset,event_wakeup,series_date_key,series_date_key_str,fold,event_pred,class_pred,class_target
0,038441c925bb,3240.0,67.174698,0.0155,0,0,0,2018-08-15,70.279156,3.235372,...,2.993552,0.0,0.0,0.0,1,038441c925bb_2018-08-15,2,0.127167,0.133527,0.0
1,038441c925bb,3241.0,68.880997,0.0210,5,0,0,2018-08-15,70.194567,3.220727,...,2.948943,0.0,0.0,0.0,1,038441c925bb_2018-08-15,2,0.000000,0.052605,0.0
2,038441c925bb,3242.0,73.113998,0.0340,10,0,0,2018-08-15,70.136580,3.214705,...,2.920777,0.0,0.0,0.0,1,038441c925bb_2018-08-15,2,0.000000,0.040674,0.0
3,038441c925bb,3243.0,73.691597,0.0304,15,0,0,2018-08-15,70.143925,3.219073,...,2.843885,0.0,0.0,0.0,1,038441c925bb_2018-08-15,2,0.000000,0.034883,0.0
4,038441c925bb,3244.0,72.684601,0.0197,20,0,0,2018-08-15,70.050436,3.210236,...,2.708060,0.0,0.0,0.0,1,038441c925bb_2018-08-15,2,0.000000,0.032879,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16520935,fe90110788d2,589315.0,48.592098,0.1187,35,59,23,2017-09-07,28.774411,21.142486,...,27.854983,-1.0,0.0,0.0,7744,fe90110788d2_2017-09-07,3,-0.000000,0.005214,-1.0
16520936,fe90110788d2,589316.0,48.228001,0.0517,40,59,23,2017-09-07,27.889183,21.266856,...,28.010473,-1.0,0.0,0.0,7744,fe90110788d2_2017-09-07,3,-0.000000,0.006759,-1.0
16520937,fe90110788d2,589317.0,6.104100,0.0089,45,59,23,2017-09-07,27.083858,21.436822,...,27.694473,-1.0,0.0,0.0,7744,fe90110788d2_2017-09-07,3,-0.000000,0.010207,-1.0
16520938,fe90110788d2,589318.0,2.872600,0.0090,50,59,23,2017-09-07,26.207230,21.661460,...,27.457500,-1.0,0.0,0.0,7744,fe90110788d2_2017-09-07,3,-0.000000,0.018845,-1.0


In [4]:
# postprocess_fn
# series_idでgroupbyして、class_predに対して対象の列のデータから前のN個の列までのデータの平均をとる
import torch
import torch.nn as nn

def detect_event_from_classpred(df, N=300, maxpool_kernel_size=41):
    df = df.copy()

    # series_idでgroupbyして、class_predに対して対象の列のデータから
    # 前のN個の列までのデータの平均をとる
    df["class_pred_beforemean"] = df.groupby("series_id")["class_pred"].apply(
        lambda x: x.rolling(N, min_periods=1).mean()
    )
    df["class_pred_aftermean"] = df.groupby("series_id")["class_pred"].apply(
        lambda x: x[::-1].rolling(N, min_periods=1).mean()[::-1]
    )
    df["event_pred"] = df["class_pred_beforemean"] - df["class_pred_aftermean"]

    # 入力サイズと出力サイズが一致するようにpaddingを調整
    maxpool_padding = int((maxpool_kernel_size - 1) / 2)
    # maxpoolしてピーク検出
    max_pooling = nn.MaxPool1d(
        maxpool_kernel_size, stride=1, padding=maxpool_padding
    )
    event_pred = df["event_pred"].values
    event_pred = torch.tensor(event_pred).unsqueeze(0)
    pooled_event_pred = max_pooling(np.abs(event_pred)).squeeze(0).numpy()
    event_pred = event_pred.squeeze(0).numpy()
    # peakのところだけ残すmaskを作成
    peak_event_pred_mask = np.where(pooled_event_pred == np.abs(event_pred), 1, 0)
    peak_event_pred = event_pred * peak_event_pred_mask
    df["event_pred"] = peak_event_pred
    df = df.drop(["class_pred_beforemean", "class_pred_aftermean"], axis=1)

    return df

def postprocess_notrolling(df):
    df = df.copy()
    # 一つ前のclass_predを取得
    df["class_pred_before"] = df.groupby("series_id")["class_pred"].shift(1)
    # 一つ後のclass_predを取得
    df["class_pred_after"] = df.groupby("series_id")["class_pred"].shift(-1)
    df["event_pred"] = df["class_pred_before"] - df["class_pred_after"]
    not_predicted_mask = (df["class_pred"] != -1).astype(int)
    df["event_pred"] = df["event_pred"] * not_predicted_mask
    df = df.drop(["class_pred_before", "class_pred_after"], axis=1)
    return df

def make_submission_df(df, threshold=0.1):
    df = df[["series_id", "step", "event_pred"]].copy()
    # thresholdより大きいときは1,-thresholdより小さいときは-1,それ以外は0
    df["event"] = df["event_pred"].apply(lambda x: 1 if x > threshold else -1 if x < -threshold else 0)
    df = df[df["event"] != 0].copy()
    df["event"] = df["event"].replace({1: "wakeup", -1: "onset"})
    df["score"] = df["event_pred"].apply(lambda x: np.clip(np.abs(x), 0.0, 1.0))
    return df


In [5]:
train_event_df = pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
train_event_df = train_event_df[train_event_df["step"].notnull()].copy()
# train_event_df = train_event_df[train_event_df["series_id"].isin(oof_df["series_id"].unique())].copy()
train_event_df = train_event_df.reset_index(drop=True)
display(train_event_df.head())


,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400


In [12]:
%%time
max_kernel_size = 3
tmp_score = 0
for average_size_ in range(60, 301, 12):
    print("average_size", average_size_, "max_kernel_size", max_kernel_size)
    df = detect_event_from_classpred(oof_df, N=average_size_, maxpool_kernel_size=max_kernel_size)
    sub_df = make_submission_df(df, threshold=0.1)
    if len(sub_df) == 0:
        print("event not detected")
        continue
    print(len(sub_df))
    score_ = score(train_event_df, sub_df)
    print(score_)
    if tmp_score > score_:
        break
    tmp_score = score_
    average_size = average_size_


average_size 12 max_kernel_size 3
17855
0.6843767998091923
average_size 24 max_kernel_size 3
20652
0.7357300907196069
average_size 36 max_kernel_size 3
21386
0.7471342410203061
average_size 48 max_kernel_size 3
21481
0.7510448376726773
average_size 60 max_kernel_size 3
21261
0.7542569081556234
average_size 72 max_kernel_size 3
20961
0.7555677910802436
average_size 84 max_kernel_size 3
20502
0.7560615289665622
average_size 96 max_kernel_size 3
20134
0.7556717560907094
CPU times: user 6min 2s, sys: 1min 36s, total: 7min 39s
Wall time: 7min 30s


In [14]:
tmp_score = 0
max_kernel_size_ = max_kernel_size
for max_kernel_size in range(23, 31, 2):
    print("average_size", average_size, "max_kernel_size", max_kernel_size)
    df = detect_event_from_classpred(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size)
    sub_df = make_submission_df(df, threshold=0.1)
    if len(sub_df) == 0:
        print("event not detected")
        continue
    score_ = score(train_event_df, sub_df)
    print(score_)
    if tmp_score > score_:
        break
    tmp_score = score_
    max_kernel_size_ = max_kernel_size


average_size 84 max_kernel_size 3
0.7560615289665622
average_size 84 max_kernel_size 5
0.7575695135735404
average_size 84 max_kernel_size 7
0.7583957383345815
average_size 84 max_kernel_size 9
0.7594686253739311
average_size 84 max_kernel_size 11
0.7602709849792437
average_size 84 max_kernel_size 13
0.7611551290637832
average_size 84 max_kernel_size 15
0.761640961576349
average_size 84 max_kernel_size 17
0.7622725980558802
average_size 84 max_kernel_size 19
0.7624328002079075
average_size 84 max_kernel_size 21
0.7626613297885241
average_size 84 max_kernel_size 23
0.7629527381799202
average_size 84 max_kernel_size 25
0.7630692918839759
average_size 84 max_kernel_size 27
0.7630600455875005


In [16]:
df = detect_event_from_classpred(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size_)
sub_df = make_submission_df(df, threshold=0.01)
print(average_size, max_kernel_size_)

if len(sub_df) == 0:
    print("event not detected")
    score_ = 0
else:
    score_ = score(train_event_df, sub_df)
print(score_)


84 25
0.7763627762772727


exp020_dense_chh_skffold_epoch10

11 3

0.7551201805225058

exp020_dense_chh_skffold_epoch30

13 3

0.7559922422713086

exp021_dense_chh_skffold_epoch30

11 3

0.7617472179453586